In [1]:
from dolfinx import default_scalar_type
from dolfinx.fem import (Constant, dirichletbc, Function, FunctionSpace, assemble_scalar,
                         form, locate_dofs_geometrical, locate_dofs_topological)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import XDMFFile, gmshio
from dolfinx.mesh import create_unit_square, locate_entities
from dolfinx.plot import vtk_mesh
from dolfinx import io

from ufl import (SpatialCoordinate, TestFunction, TrialFunction,
                 dx, grad, inner)

from mpi4py import MPI

import meshio
import gmsh
import numpy as np
import pyvista
##########################################################################################################3

  

In [2]:
if prune_z:
    points = mesh.points[:, :3]
else:
    points = mesh.points

NameError: name 'prune_z' is not defined

In [7]:
## from official tutorial
def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:, :3] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={"name_to_read": [cell_data.astype(np.int32)]})
    return out_mesh

In [8]:
# mesh file is helix1.msh
mesh, cell_markers, facet_markers = gmshio.read_from_msh("helix2.msh", MPI.COMM_WORLD, gdim=3)

Info    : Reading 'helix2.msh'...
Info    : 7982 nodes
Info    : 39006 elements
Info    : Done reading 'helix2.msh'


In [9]:
## cell type: tetrahedron and triangle is choosen in Salome, which step before generate .mesh file.
mesh = meshio.read("helix2.msh")
meshio.write("helix.xdmf", create_mesh(mesh, "tetra", True))

volume_mesh = create_mesh(mesh, "tetra", prune_z=True)
face_mesh = create_mesh(mesh, "triangle", prune_z=True)
meshio.write("mesh.xdmf", volume_mesh)
meshio.write("mt.xdmf", face_mesh)

In [10]:
with XDMFFile(MPI.COMM_WORLD, "mesh.xdmf", "r") as xdmf:
    mesh = xdmf.read_mesh(name="Grid")
    cd = xdmf.read_meshtags(mesh, name="Grid")    # volume mesh
mesh.topology.create_connectivity(mesh.topology.dim, mesh.topology.dim - 1)
with XDMFFile(MPI.COMM_WORLD, "mt.xdmf", "r") as xdmf:
    fd = xdmf.read_meshtags(mesh, name="Grid")    # face element

In [11]:
print(mesh)
print(fd)
print(cd)

In [ ]:
# https://fenicsproject.discourse.group/t/can-dolfinx-read-a-gmsh-file-containing-1d-mesh-of-an-ellipse/9765/2